## Modifying MINE Approach for CMI estimation

In [9]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

### Data Generating Mechanism

In [63]:
def generate_data(num_samples, num_features, correlation_strength):
  X = np.random.randn(num_samples, num_features)
  Y = np.random.randn(num_samples, num_features) + X * correlation_strength
  Z = np.random.randn(num_samples, num_features)
  return X, Y, Z

### Basic NN Model for Neural Separation

In [12]:
def build_and_train_model(X, Y, Z):
  # Concatenate X and Z
  XZ = np.concatenate([X, Z], axis=1)

  # Create positive and negative samples
  positive_samples = np.concatenate([XZ, Y], axis=1)
  negative_samples = np.concatenate([XZ, np.random.permutation(Y)], axis=1)

  # Create a dataset
  dataset = tf.data.Dataset.from_tensor_slices((positive_samples, np.ones(len(positive_samples)))).concatenate(
      tf.data.Dataset.from_tensor_slices((negative_samples, np.zeros(len(negative_samples))))).shuffle(buffer_size=1024).batch(32)

  # Build the model
  model = Sequential([
      Dense(64, activation='relu', input_dim=positive_samples.shape[1]),
      Dense(32, activation='relu'),
      Dense(1)
  ])

  # Compile the model
  model.compile(loss='binary_crossentropy', optimizer='adam')

  # Train the model
  model.fit(dataset, epochs=10, verbose=1)

  return model

In [13]:
def estimate_conditional_mi(model, X, Y, Z):
  # Concatenate X and Z
  XZ = np.concatenate([X, Z], axis=1)

  # Create positive and negative samples
  positive_samples = np.concatenate([XZ, Y], axis=1)
  negative_samples = np.concatenate([XZ, np.random.permutation(Y)], axis=1)

  # Predict probabilities for positive and negative samples
  positive_scores = model.predict(positive_samples)
  negative_scores = model.predict(negative_samples)

  # Estimate CMI using a simple difference in means
  estimated_cmi = np.mean(positive_scores) - np.mean(negative_scores)

  return estimated_cmi

In [14]:
num_samples = 1000
num_features = 5
correlation_strength = 0.5

X, Y, Z = generate_data(num_samples, num_features, correlation_strength)

model = build_and_train_model(X, Y, Z)

estimated_cmi = estimate_conditional_mi(model, X, Y, Z)

print("Estimated Conditional Mutual Information:", estimated_cmi)

Epoch 1/10


2024-12-06 22:09:14.566531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [1000,15]
	 [[{{node Placeholder/_3}}]]
2024-12-06 22:09:14.566774: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [1000]
	 [[{{node Placeholder/_4}}]]
2024-12-06 22:09:14.773772: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 10ms/step - loss: 5.7540
Epoch 2/10
63/63 [==============================] - 1s 8ms/step - loss: 4.6240
Epoch 3/10
63/63 [==============================] - 1s 8ms/step - loss: 3.8947
Epoch 4/10
63/63 [==============================] - 0s 8ms/step - loss: 2.4510
Epoch 5/10
63/63 [==============================] - 0s 8ms/step - loss: 0.9573
Epoch 6/10
63/63 [==============================] - 0s 8ms/step - loss: 0.9066
Epoch 7/10
63/63 [==============================] - 0s 8ms/step - loss: 0.7094
Epoch 8/10
63/63 [==============================] - 1s 8ms/step - loss: 0.7035
Epoch 9/10
63/63 [==============================] - 0s 8ms/step - loss: 0.7025
Epoch 10/10
32/32 [==============================] - 0s 2ms/step
Estimated Conditional Mutual Information: -2.9802322e-08


2024-12-06 22:09:20.222393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


### Complete Workflow 
I have modified the workflow so as to be able to modularly call and iterate over different input values

In [24]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_mine_model(input_shape):
  model = Sequential([
      Dense(64, activation='relu', input_shape=(input_shape,)),  # Wrap num_features in a tuple
      Dense(32, activation='relu'),
      Dense(1)
  ])
  model.compile(loss='binary_crossentropy', optimizer='adam')
  return model

def estimate_conditional_mi(X, Y, Z):
  # Concatenate X and Z
  XZ = np.concatenate([X, Z], axis=1)

  # Create positive and negative samples
  positive_samples = np.concatenate([XZ, Y], axis=1)
  negative_samples = np.concatenate([XZ, np.random.permutation(Y)], axis=1)

  # Create a dataset
  dataset = tf.data.Dataset.from_tensor_slices((positive_samples, np.ones(len(positive_samples)))).concatenate(
      tf.data.Dataset.from_tensor_slices((negative_samples, np.zeros(len(negative_samples))))).shuffle(buffer_size=1024).batch(32)

  # Build and train the MINE model
  mine_model = build_mine_model(positive_samples.shape[1])
  mine_model.fit(dataset, epochs=10, verbose=1)

  # Estimate I(X,Z;Y)
  I_XZ_Y = mine_model.predict(positive_samples).mean() - mine_model.predict(negative_samples).mean()

  # Estimate I(Z;Y)
  ZY = np.concatenate([Z, Y], axis=1)
  negative_ZY = np.concatenate([Z, np.random.permutation(Y)], axis=1)
  
  dataset_ZY = tf.data.Dataset.from_tensor_slices((ZY, np.ones(len(ZY)))).concatenate(
      tf.data.Dataset.from_tensor_slices((negative_ZY, np.zeros(len(negative_ZY))))).shuffle(buffer_size=1024).batch(32)

  mine_model_ZY = build_mine_model(ZY.shape[1])
  mine_model_ZY.fit(dataset_ZY, epochs=10, verbose=1)

  I_Z_Y = mine_model_ZY.predict(ZY).mean() - mine_model_ZY.predict(negative_ZY).mean()

  # Estimate conditional mutual information
  conditional_mi = I_XZ_Y - I_Z_Y

  return conditional_mi

# Example usage:
num_samples = 1000
num_features = 5
correlation_strength = 0.5

X, Y, Z = generate_data(num_samples, num_features, correlation_strength)
estimated_cmi = estimate_conditional_mi(X, Y, Z)

print("Estimated Conditional Mutual Information:", estimated_cmi)

Epoch 1/10


2024-12-06 22:25:15.095145: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1000,15]
	 [[{{node Placeholder/_0}}]]
2024-12-06 22:25:15.095379: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [1000,15]
	 [[{{node Placeholder/_0}}]]
2024-12-06 22:25:15.260357: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 10ms/step - loss: 4.9313
Epoch 2/10
63/63 [==============================] - 0s 8ms/step - loss: 4.3654
Epoch 3/10
63/63 [==============================] - 1s 8ms/step - loss: 3.3031
Epoch 4/10
63/63 [==============================] - 1s 9ms/step - loss: 2.5292
Epoch 5/10
63/63 [==============================] - 1s 10ms/step - loss: 0.8484
Epoch 6/10
63/63 [==============================] - 1s 9ms/step - loss: 0.7109
Epoch 7/10
63/63 [==============================] - 1s 9ms/step - loss: 0.7038
Epoch 8/10
63/63 [==============================] - 1s 9ms/step - loss: 0.7010
Epoch 9/10
63/63 [==============================] - 1s 9ms/step - loss: 0.7036
Epoch 10/10
32/32 [==============================] - 0s 2ms/step


2024-12-06 22:25:21.099348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


Epoch 1/10


2024-12-06 22:25:21.311060: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [1000]
	 [[{{node Placeholder/_4}}]]
2024-12-06 22:25:21.311290: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [1000]
	 [[{{node Placeholder/_4}}]]
2024-12-06 22:25:21.482262: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


63/63 [==============================] - 1s 10ms/step - loss: 4.1683
Epoch 2/10
63/63 [==============================] - 1s 9ms/step - loss: 3.2254
Epoch 3/10
63/63 [==============================] - 1s 8ms/step - loss: 1.0109
Epoch 4/10
63/63 [==============================] - 1s 8ms/step - loss: 0.7110
Epoch 5/10
63/63 [==============================] - 1s 8ms/step - loss: 0.7037
Epoch 6/10
63/63 [==============================] - 1s 9ms/step - loss: 0.7027
Epoch 7/10
63/63 [==============================] - 0s 8ms/step - loss: 0.7008
Epoch 8/10
63/63 [==============================] - 0s 8ms/step - loss: 0.7011
Epoch 9/10
63/63 [==============================] - 0s 8ms/step - loss: 0.7006
Epoch 10/10
32/32 [==============================] - 0s 2ms/step
Estimated Conditional Mutual Information: 0.0


2024-12-06 22:25:27.044503: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [26]:
# Define ranges for hyperparameters
num_samples_range = [100, 1000, 10000]
num_features_range = [5, 10, 50, 100, 200]  # Increased range for higher dimensions
correlation_strength_range = [0.2]

# Results dictionary
results = {}

# Loop through all combinations
for num_samples in num_samples_range:
    for num_features in num_features_range:
        for correlation_strength in correlation_strength_range:
            key = (num_samples, num_features, correlation_strength)
            X, Y, Z = generate_data(num_samples, num_features, correlation_strength)
            estimated_cmi = estimate_conditional_mi(X, Y, Z)
            results[key] = estimated_cmi

# Print the results in a tabular format
print("Results:")
print("|{:15s}|{:15s}|{:15s}|{:25s}|".format('Num Samples', 'Num Features', 'Corr Strength', 'CMI Estimate'))
print("-"*80)
for key, value in results.items():
    num_samples, num_features, correlation_strength = key
    print("|{:15d}|{:15d}|{:15.2f}|{:25.8f}|".format(num_samples, num_features, correlation_strength, value))

Epoch 1/10


2024-12-06 22:29:12.363566: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [100,15]
	 [[{{node Placeholder/_3}}]]
2024-12-06 22:29:12.363803: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [100,15]
	 [[{{node Placeholder/_0}}]]
2024-12-06 22:29:12.535935: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 41ms/step - loss: 5.2008
Epoch 2/10
7/7 [==============================] - 0s 13ms/step - loss: 5.1025
Epoch 3/10
7/7 [==============================] - 0s 13ms/step - loss: 4.5978
Epoch 4/10
7/7 [==============================] - 0s 12ms/step - loss: 4.5027
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 4.4405
Epoch 6/10
7/7 [==============================] - 0s 16ms/step - loss: 4.3749
Epoch 7/10
7/7 [==============================] - 0s 9ms/step - loss: 4.3578
Epoch 8/10
7/7 [==============================] - 0s 10ms/step - loss: 4.2655
Epoch 9/10
7/7 [==============================] - 0s 11ms/step - loss: 4.1108
Epoch 10/10
4/4 [==============================] - 0s 2ms/step
Epoch 1/10


2024-12-06 22:29:13.906652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-06 22:29:14.013549: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [100,10]
	 [[{{node Placeholder/_3}}]]
2024-12-06 22:29:14.013789: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [100,10]
	 [[{{node Placeholder/_0}}]]
2024-12-06 22:29:14.182463: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled

7/7 [==============================] - 1s 40ms/step - loss: 5.2257
Epoch 2/10
7/7 [==============================] - 0s 16ms/step - loss: 4.8891
Epoch 3/10
7/7 [==============================] - 0s 11ms/step - loss: 4.5543
Epoch 4/10
7/7 [==============================] - 0s 8ms/step - loss: 4.2270
Epoch 5/10
7/7 [==============================] - 0s 8ms/step - loss: 4.1201
Epoch 6/10
7/7 [==============================] - 0s 9ms/step - loss: 4.0727
Epoch 7/10
7/7 [==============================] - 0s 11ms/step - loss: 4.1738
Epoch 8/10
7/7 [==============================] - 0s 10ms/step - loss: 3.9725
Epoch 9/10
7/7 [==============================] - 0s 9ms/step - loss: 3.6799
Epoch 10/10
4/4 [==============================] - 0s 2ms/step
Epoch 1/10


...

313/313 [==============================] - 1s 2ms/step
Epoch 1/10


2024-12-06 22:39:13.663414: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype double and shape [10000]
	 [[{{node Placeholder/_4}}]]
2024-12-06 22:39:13.663740: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [10000,400]
	 [[{{node Placeholder/_3}}]]
2024-12-06 22:39:13.834139: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 5s 8ms/step - loss: 7.1101
Epoch 2/10
625/625 [==============================] - 5s 7ms/step - loss: 7.0677
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 7.2869
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 7.2213
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 7.3136
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 7.3073
Epoch 7/10
625/625 [==============================] - 5s 7ms/step - loss: 7.2806
Epoch 8/10
625/625 [==============================] - 5s 7ms/step - loss: 7.2877
Epoch 9/10
625/625 [==============================] - 5s 7ms/step - loss: 7.3051
Epoch 10/10
100/313 [========>.....................] - ETA: 0s

2024-12-06 22:40:02.141508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 0s 2ms/step
Results:
|Num Samples    |Num Features   |Corr Strength  |CMI Estimate             |
--------------------------------------------------------------------------------
|            100|              5|           0.20|              -0.00000002|
|            100|             10|           0.20|               0.00000000|
|            100|             50|           0.20|               0.00000004|
|            100|            100|           0.20|              -0.00000010|
|            100|            200|           0.20|              -0.00000025|
|           1000|              5|           0.20|               0.00000003|
|           1000|             10|           0.20|               0.00000003|
|           1000|             50|           0.20|              -0.00000001|
|           1000|            100|           0.20|               0.00000008|
|           1000|            200|           0.20|              -0.00000004|
|          10000|  

Testing for correlation strength of 0.8

In [60]:
# Define ranges for hyperparameters
num_samples_range = [100, 1000, 10000]
num_features_range = [5, 10, 50, 100, 200]  
correlation_strength_range = [0.5, 0.8]

# Results dictionary
results = {}

# Loop through all combinations
for num_samples in num_samples_range:
    for num_features in num_features_range:
        for correlation_strength in correlation_strength_range:
            key = (num_samples, num_features, correlation_strength)
            X, Y, Z = generate_data(num_samples, num_features, correlation_strength)
            estimated_cmi = estimate_conditional_mi(X, Y, Z)
            results[key] = estimated_cmi*(10000000)

# Print the results in a tabular format
print("Results:")
print("|{:15s}|{:15s}|{:15s}|{:25s}|".format('Num Samples', 'Num Features', 'Corr Strength', 'CMI Estimate'))
print("-"*80)
for key, value in results.items():
    num_samples, num_features, correlation_strength = key
    print("|{:15d}|{:15d}|{:15.2f}|{:25.8f}|".format(num_samples, num_features, correlation_strength, value))

Epoch 1/10


2024-12-09 21:33:47.196864: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [100]
	 [[{{node Placeholder/_1}}]]
2024-12-09 21:33:47.197115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [100,15]
	 [[{{node Placeholder/_0}}]]
2024-12-09 21:33:47.376281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 63ms/step - loss: 4.4884
Epoch 2/10
7/7 [==============================] - 0s 27ms/step - loss: 4.1984
Epoch 3/10
7/7 [==============================] - 0s 15ms/step - loss: 3.8879
Epoch 4/10
7/7 [==============================] - 0s 12ms/step - loss: 4.0022
Epoch 5/10
7/7 [==============================] - 0s 10ms/step - loss: 3.4226
Epoch 6/10
7/7 [==============================] - 0s 10ms/step - loss: 3.3674
Epoch 7/10
7/7 [==============================] - 0s 9ms/step - loss: 3.2933
Epoch 8/10
7/7 [==============================] - 0s 14ms/step - loss: 2.8016
Epoch 9/10
7/7 [==============================] - 0s 11ms/step - loss: 2.4758
Epoch 10/10
4/4 [==============================] - 0s 3ms/step
Epoch 1/10


2024-12-09 21:33:49.158756: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-12-09 21:33:49.274713: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [100]
	 [[{{node Placeholder/_1}}]]
2024-12-09 21:33:49.274954: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [100,10]
	 [[{{node Placeholder/_0}}]]
2024-12-09 21:33:49.436879: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


7/7 [==============================] - 1s 60ms/step - loss: 5.4500
Epoch 2/10
7/7 [==============================] - 0s 20ms/step - loss: 5.1659
Epoch 3/10
7/7 [==============================] - 0s 20ms/step - loss: 4.9144
Epoch 4/10
7/7 [==============================] - 0s 14ms/step - loss: 4.7101
Epoch 5/10
7/7 [==============================] - 0s 9ms/step - loss: 4.3244
Epoch 6/10
7/7 [==============================] - 0s 14ms/step - loss: 4.6001
Epoch 7/10
7/7 [==============================] - 0s 10ms/step - loss: 4.4827
Epoch 8/10
7/7 [==============================] - 0s 11ms/step - loss: 4.2810
Epoch 9/10
7/7 [==============================] - 0s 8ms/step - loss: 3.8264
Epoch 10/10
4/4 [==============================] - 0s 3ms/step
Epoch 1/10


...

313/313 [==============================] - 1s 2ms/step
Epoch 1/10


2024-12-09 22:30:16.537121: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_3' with dtype double and shape [10000,400]
	 [[{{node Placeholder/_3}}]]
2024-12-09 22:30:16.537381: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype double and shape [10000]
	 [[{{node Placeholder/_1}}]]
2024-12-09 22:30:16.712006: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 6s 9ms/step - loss: 7.0839
Epoch 2/10
625/625 [==============================] - 5s 8ms/step - loss: 7.2075
Epoch 3/10
625/625 [==============================] - 5s 8ms/step - loss: 7.1340
Epoch 4/10
625/625 [==============================] - 5s 8ms/step - loss: 7.1919
Epoch 5/10
625/625 [==============================] - 5s 8ms/step - loss: 7.3289
Epoch 6/10
625/625 [==============================] - 5s 8ms/step - loss: 7.4599
Epoch 7/10
625/625 [==============================] - 5s 8ms/step - loss: 7.4125
Epoch 8/10
625/625 [==============================] - 5s 8ms/step - loss: 7.4737
Epoch 9/10
625/625 [==============================] - 5s 8ms/step - loss: 7.5049
Epoch 10/10
 88/313 [=======>......................] - ETA: 0s

2024-12-09 22:31:09.294797: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 1s 2ms/step
Results:
|Num Samples    |Num Features   |Corr Strength  |CMI Estimate             |
--------------------------------------------------------------------------------
|            100|              5|           0.50|               0.29802322|
|            100|              5|           0.80|               0.96857548|
|            100|             10|           0.50|               0.74505806|
|            100|             10|           0.80|               0.07450581|
|            100|             50|           0.50|              -0.26077032|
|            100|             50|           0.80|               2.60770321|
|            100|            100|           0.50|               0.14901161|
|            100|            100|           0.80|               1.78813934|
|            100|            200|           0.50|              -2.11410224|
|            100|            200|           0.80|              -0.37252903|
|           1000|  

We can clearly see that the trendline is as expected when we scale up the difference by 10^6. This is implying that something is going wrong in either data generating mechanism or there is a possibility of insufficient training

In [62]:
import pandas as pd
df = pd.DataFrame(results.items())
df.to_csv("Results.csv")

### Tried to implement and compare with HSIC (Could ignore)

In [66]:
from hyppo.independence import Hsic

def estimate_conditional_mi_hsic(X, Y, Z):
  # Concatenate X and Z
  XZ = np.concatenate([X, Z], axis=1)

  # Create Hsic object
  hsic = Hsic(compute_kernel='gaussian', bias=False)

  # Calculate HSIC statistic for XZ and Y
  hsic_xz_y_result = hsic.test(XZ, Y)
  hsic_xz_y = hsic_xz_y_result.stat

  # Calculate HSIC between Z and Y
  ZY = np.concatenate([Z, Y], axis=1)
  hsic_z_y_result = hsic.test(ZY, ZY)  # Pass ZY as both X and Y
  hsic_z_y = hsic_z_y_result.stat

  # Estimate conditional mutual information
  conditional_mi = hsic_xz_y - hsic_z_y

  return conditional_mi

In [65]:
# Define ranges for hyperparameters
num_samples_range = [100, 1000, 10000]
num_features_range = [5, 10, 50, 100, 200]  # Increased range for higher dimensions
correlation_strength_range = [0.2]

# Results dictionary
results_hsic = {}

# Loop through all combinations
for num_samples in num_samples_range:
    for num_features in num_features_range:
        for correlation_strength in correlation_strength_range:
            key = (num_samples, num_features, correlation_strength)
            X, Y, Z = generate_data(num_samples, num_features, correlation_strength)
            estimated_cmi = estimate_conditional_mi_hsic(X, Y, Z)
            results[key] = estimated_cmi

# Print the results in a tabular format
print("Results:")
print("|{:15s}|{:15s}|{:15s}|{:25s}|".format('Num Samples', 'Num Features', 'Corr Strength', 'CMI Estimate'))
print("-"*80)
for key, value in results_hsic.items():
    num_samples, num_features, correlation_strength = key
    print("|{:15d}|{:15d}|{:15.2f}|{:25.8f}|".format(num_samples, num_features, correlation_strength, value))

KeyboardInterrupt: 